# Long Short Term Memory RNN

In [49]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from  sklearn.preprocessing import  LabelEncoder,  MinMaxScaler

In [55]:
df = pd.read_csv("../data/training_data.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,d_volatility,VIX
0,2017-01-03,2251.570068,2263.879883,2245.129883,2257.830078,2257.830078,3770530000,0.008328,12.85
1,2017-01-04,2261.600098,2272.820068,2261.600098,2270.750000,2270.750000,3764890000,0.004961,11.85
2,2017-01-05,2268.179932,2271.500000,2260.449951,2269.000000,2269.000000,3761820000,0.004872,11.67
3,2017-01-06,2271.139893,2282.100098,2264.060059,2276.979980,2276.979980,3339890000,0.007943,11.32
4,2017-01-09,2273.590088,2275.489990,2268.899902,2268.899902,2268.899902,3217610000,0.002899,11.56


# Pre procesamiento

In [56]:
df = df.set_index('Date')
df = df[['Adj Close', "Volume", "d_volatility", "VIX"]]
df.head()

,Adj Close,Volume,d_volatility,VIX
Date,,,,
2017-01-03,2257.830078,3770530000,0.008328,12.85
2017-01-04,2270.750000,3764890000,0.004961,11.85
2017-01-05,2269.000000,3761820000,0.004872,11.67
2017-01-06,2276.979980,3339890000,0.007943,11.32
2017-01-09,2268.899902,3217610000,0.002899,11.56


## Train y Test

In [65]:
len(df)*0.75


935.25

In [66]:
train_df,test_df = df[1:935], df[935:] 

## Escalando los datos

In [67]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/var/folders/l4/br6xdwsn3wj7ntybwy6zwmvh0000gn/T/ipykernel_33029/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
/var/folders/l4/br6xdwsn3wj7ntybwy6zwmvh0000gn/T/ipykernel_33029/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
/var/folders/l4/br6xdwsn3wj7ntybwy6zwmvh0000gn/T/ipykernel_33029/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [69]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

## Defino ccc

In [72]:
n_past = 32
n_future = 16
n_features = 4

In [73]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [75]:
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(50,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(50, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 4)]      0           []                               
                                                                                                  
 lstm_1 (LSTM)                  [(None, 32, 100),    42000       ['input_1[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_2 (LSTM)                  [(None, 100),        80400       ['lstm_1[0][0]']                 
                                 (None, 100),                                                 

In [76]:
    #definir "early stopping" la loss de validacion no mejora en X épocas
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=20,  #numero de epocas sin mejora
                                                      restore_best_weights = True)
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[early_stopping])

2022-01-22 18:11:39.426575: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-22 18:11:40.805401: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:11:41.219256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:11:41.543438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:11:41.670125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:11:41.738239: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:11:42.746822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112

In [77]:
pred_e2d2=model_e2d2.predict(X_test)
pred_e2d2

2022-01-22 18:12:39.316591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:12:39.443055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:12:39.501637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:12:39.564147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-22 18:12:39.603172: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[[ 4.85175967e-01, -4.39756401e-02, -5.02034545e-01,
         -4.42056447e-01],
        [ 4.25293952e-01, -1.39818164e-02, -5.08318245e-01,
         -4.24052835e-01],
        [ 3.91873449e-01, -9.18882061e-03, -5.43931901e-01,
         -4.18386132e-01],
        ...,
        [ 6.67697310e-01, -8.47237110e-02, -8.52339268e-01,
         -5.07922590e-01],
        [ 6.69291079e-01, -7.63325468e-02, -8.53782415e-01,
         -5.11883795e-01],
        [ 6.73714221e-01, -7.10346475e-02, -8.60993147e-01,
         -5.17308831e-01]],

       [[ 5.00328541e-01, -3.36195342e-02, -4.95921344e-01,
         -4.33069617e-01],
        [ 4.39497411e-01, -8.07163306e-04, -4.98444349e-01,
         -4.11962539e-01],
        [ 4.02719140e-01,  6.01294171e-03, -5.29653609e-01,
         -4.03213173e-01],
        ...,
        [ 6.32003248e-01, -6.48048222e-02, -8.20303798e-01,
         -4.79433000e-01],
        [ 6.34368777e-01, -5.67468964e-02, -8.22296143e-01,
         -4.82266992e-01],
        [ 6.383

Escalado inverso

In [78]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [79]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
  print(i)
  for j in range(1,6):
    print("Day ",j,":")
    print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index]))
  print()
  print()

Adj Close
Day  1 :
MAE-E2D2 :  640.9709408166274
Day  2 :
MAE-E2D2 :  691.2287146226415
Day  3 :
MAE-E2D2 :  729.1224747567807
Day  4 :
MAE-E2D2 :  742.2270627579599
Day  5 :
MAE-E2D2 :  737.768539946934


Volume
Day  1 :
MAE-E2D2 :  906986612.3471698
Day  2 :
MAE-E2D2 :  1026620451.4415095
Day  3 :
MAE-E2D2 :  1056433085.7056603
Day  4 :
MAE-E2D2 :  1061555480.8150944
Day  5 :
MAE-E2D2 :  1044037151.0339622


d_volatility
Day  1 :
MAE-E2D2 :  0.004212867577257586
Day  2 :
MAE-E2D2 :  0.004945165764668399
Day  3 :
MAE-E2D2 :  0.00504133114381691
Day  4 :
MAE-E2D2 :  0.004767696933826661
Day  5 :
MAE-E2D2 :  0.0045192007986409564


VIX
Day  1 :
MAE-E2D2 :  2.2282134480026534
Day  2 :
MAE-E2D2 :  3.5594380282995832
Day  3 :
MAE-E2D2 :  4.1861555480957024
Day  4 :
MAE-E2D2 :  4.3633789583602045
Day  5 :
MAE-E2D2 :  4.307232117850826


